# Ufuncs et al

## ufunc in its simplest form


It feels similar in spirit but it's different things conceptually. I am talking about the universal functions and broadcasting. They are similar in spirit because both are designed to avoid loops as much as possible, but ufuncs are dealing with elementwise operations and broadcast is dealing with operations across arrays of different shapes and sizes.

The implementing deals are underwhelming as long as one is not interested in customizing it. (Customizing it usually yeilds slow results and we might need to use numba to do acceleration; more on that later). However, conceptually, there is unary ufunc and binary ufuncs. The main difference is unary takes one (array) and spits one out (just think about exponentiate operations), but binary ufuncs takes two in and spits one out (just think about add, subtract, multiple divide, you need two array to finish that kind of operations). 

In [21]:
import numpy as np 
np.random.seed(123)
unary_ufunc_sign = np.sign(np.random.randn(10))
unary_ufunc_ceil = np.ceil(np.random.randn(10))
unary_ufunc_floor = np.floor(np.random.randn(10))
frac, int_ = np.modf(np.random.randn(10))

print(unary_ufunc_ceil, unary_ufunc_sign, unary_ufunc_floor, frac)

[-0. -0.  2. -0. -0. -0.  3.  3.  2.  1.] [-1.  1.  1. -1. -1.  1. -1. -1.  1. -1.] [ 0.  1. -1.  1. -2. -1.  0. -2. -1. -1.] [-0.25561937 -0.79858911 -0.7715331  -0.69987723  0.92746243 -0.17363568
  0.00284592  0.68822271 -0.87953634  0.28362732]


In [25]:
binary_ufunc = np.copysign(np.random.randn(10),  np.random.randn(10))
print(binary_ufunc)
binary_ufunc = np.not_equal(np.random.randn(10),  2)
print(binary_ufunc)

[-0.99732147  1.10004311  0.75643721  0.32168658  0.76094939 -0.32346885
  0.5489551   1.80597011 -1.51886562  0.35400011]
[ True  True  True  True  True  True  True  True  True  True]


Above are some lesser known functions in numpy. sign takes the sign of input and has three possible values, -1, 1 and 0. ceil is to get the smallest int that is larger than x, while floor is to get the largest int that smaller than that number. (Just think what a floor and ceil is.) modf is one of the few (maybe only) ufunc that returns two arrays because it splits each element in the array into the whole part and the fractional part.

On the other hand, the binary ufunc family also has some lesser known members, copysign, for instance, copies the sign of the second arg to the first arg. And np.not_equal is literally an logical operator. It is less know because one can simply use np.random.randn(10)!=2 and get the same result in most use cases.

## now let's cumtomize ufunc and make it faster

Again, ufunc is desiged to do elementwise operations, and binary ufunc operation on two arrays while unary ufunc operates on one array. Hence, we have to define a elementwise function before we package it as a ufunc

In [27]:
def simple_square(x):
    return x**2
square_all = np.frompyfunc(simple_square, 1, 1) # number of args to take unary=1 binary=2

In [40]:
%%timeit
result = square_all(np.arange(1000))
result.astype(np.int64)

330 µs ± 3.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [32]:
%%timeit
result = np.square(np.arangeresult(1000))

2.02 µs ± 40.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Two things:

(1) The purpose of this function is to mimic the behavior of ufunc but cannot hold a candle to optimimzed built in ufuncs. As one can see here the speed is two magnitude worse compared to native ufuncs. 
(2) Another drawback is if we print dype of the result, it is ('O') although we passed only int into the function. That's why we got to cast the array to integers after it done. To avoid this issue, one can use np.vectorize alternatively. Does not help with speed tho.

In [44]:
%%timeit
square_vector = np.vectorize(simple_square, otypes=[int])
result = square_vector(np.arange(1000))

352 µs ± 4.35 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In order to get performance boost, we have to use numba package. The api is as simple as adding a decorator to the original elementwise function. 

In [45]:
from numba import vectorize
@vectorize
def simple_square(x):
    return x**2

In [47]:
%%timeit
simple_square(np.arange(1000))

1.76 µs ± 25.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


The awesome thing is this function is even after than native np.square function! My experience with numba is, it can work with only limited number of scenarios, but if it works, it is really awesome.